<a href="https://colab.research.google.com/github/GodishalaAshwith/MachineLearningLab/blob/main/MLInternal1Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ================== ALL IMPORTS ==================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

from sklearn import datasets
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, label_binarize
from sklearn.feature_selection import VarianceThreshold, SelectKBest, chi2, f_classif, f_regression, mutual_info_classif, mutual_info_regression, RFE, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge, ElasticNet
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree, export_text
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# ================== DATASETS ==================
iris = datasets.load_iris(as_frame=True); X_iris, y_iris = iris.data, iris.target
diab = datasets.load_diabetes(as_frame=True); X_diab, y_diab = diab.data, diab.target
canc = datasets.load_breast_cancer(as_frame=True); X_canc, y_canc = canc.data, canc.target

# ================== Q1 FEATURE SELECTION ==================
print("\n=== Q1 Feature Selection ===")
# 1.1 Constant
vt = VarianceThreshold(0.0); vt.fit(X_iris)
print("Non-constant iris cols:", X_iris.columns[vt.get_support()])
# 1.2 Quasi-constant
vt2 = VarianceThreshold(0.01); print("Quasi-constant mask:", vt2.fit(X_iris).get_support())
# 1.3 Duplicate
def duplicates(df):
    dup=[]; cols=df.columns
    for i in range(len(cols)):
        for j in range(i+1,len(cols)):
            if df[cols[i]].equals(df[cols[j]]): dup.append((cols[i], cols[j]))
    return dup
print("Duplicates (iris):", duplicates(X_iris))
# 1.4 Correlation
corr = X_diab.corr().abs(); upper = corr.where(np.triu(np.ones(corr.shape),1).astype(bool))
to_drop=[c for c in upper.columns if any(upper[c]>0.95)]
print("Diabetes drop cols:", to_drop)
# 1.5 MI
print("Iris MI:", dict(zip(X_iris.columns, mutual_info_classif(X_iris, y_iris))))
print("Diab MI:", dict(zip(X_diab.columns, mutual_info_regression(X_diab, y_diab))))
# 1.6 Chi2
Xmm = MinMaxScaler().fit_transform(X_iris)
chi = SelectKBest(chi2,k=2).fit(Xmm,y_iris)
print("Chi2 selected indices:", chi.get_support(indices=True))
# 1.7 ANOVA / F-regression
print("ANOVA iris:", X_iris.columns[SelectKBest(f_classif,2).fit(X_iris,y_iris).get_support()])
print("F-reg diab:", X_diab.columns[SelectKBest(f_regression,3).fit(X_diab,y_diab).get_support()])
# 1.8 ROC-AUC & MSE
y_bin = label_binarize(y_iris, classes=np.unique(y_iris))
aucs={}
for col in X_iris.columns:
    try:
        aucs[col]=np.mean([roc_auc_score(y_bin[:,c],X_iris[col]) for c in range(y_bin.shape[1])])
    except: aucs[col]=np.nan
print("Iris AUC:", aucs)
mses={}
for col in X_diab.columns:
    lr=LinearRegression().fit(X_diab[[col]],y_diab)
    mses[col]=mean_squared_error(y_diab, lr.predict(X_diab[[col]]))
print("Diab MSE:", mses)

# ================== Q2 WRAPPER METHODS ==================
print("\n=== Q2 Wrapper Methods ===")
Xtr,Xte,ytr,yte=train_test_split(X_iris,y_iris,test_size=0.3,random_state=0)
est=LogisticRegression(max_iter=500)
# Forward
sfs_fwd=SequentialFeatureSelector(est,n_features_to_select=2,direction='forward').fit(Xtr,ytr)
print("Forward:", Xtr.columns[sfs_fwd.get_support()])
# Backward
sfs_bwd=SequentialFeatureSelector(est,n_features_to_select=2,direction='backward').fit(Xtr,ytr)
print("Backward:", Xtr.columns[sfs_bwd.get_support()])
# Exhaustive
def exhaustive(X,y,model,kmax=3):
    best,bs=-1,None
    for k in range(1,min(kmax,len(X.columns))+1):
        for combo in itertools.combinations(X.columns,k):
            score=np.mean(cross_val_score(model,X[list(combo)],y,cv=3))
            if score>best: best,bs=score,combo
    return bs,best
print("Exhaustive best:", exhaustive(X_iris,y_iris,LogisticRegression(max_iter=500),3))
# RFE
rfe=RFE(estimator=LogisticRegression(max_iter=500),n_features_to_select=2).fit(Xtr,ytr)
print("RFE ranking:", dict(zip(Xtr.columns, rfe.ranking_)))

# ================== Q3 EMBEDDED METHODS ==================
print("\n=== Q3 Embedded Methods ===")
lasso=Lasso(0.1).fit(X_diab,y_diab); ridge=Ridge(1.0).fit(X_diab,y_diab); en=ElasticNet(0.1,l1_ratio=0.5).fit(X_diab,y_diab)
print("Lasso coefs:", lasso.coef_); print("Ridge coefs:", ridge.coef_); print("EN coefs:", en.coef_)
rf=RandomForestClassifier(n_estimators=50).fit(X_canc,y_canc)
print("RF top5:", sorted(zip(X_canc.columns,rf.feature_importances_),key=lambda x:-x[1])[:5])
dt=DecisionTreeClassifier().fit(X_canc,y_canc)
print("DT top5:", sorted(zip(X_canc.columns,dt.feature_importances_),key=lambda x:-x[1])[:5])
sv=SVC(kernel='linear').fit(X_canc,y_canc)
print("SVC coef abs top5:", sorted(zip(X_canc.columns,np.abs(sv.coef_.ravel())),key=lambda x:-x[1])[:5])

# ================== Q4 PCA ==================
print("\n=== Q4 PCA ===")
Xs=StandardScaler().fit_transform(X_diab); pca=PCA(n_components=10); Xp=pca.fit_transform(Xs)
plt.scatter(Xp[:,0],Xp[:,1],s=8); plt.xlabel("PC1"); plt.ylabel("PC2"); plt.title("PCA PC1 vs PC2"); plt.show()
loadings=pd.DataFrame(pca.components_,columns=X_diab.columns)
print("PC1 top:", loadings.iloc[0].abs().sort_values(ascending=False).head())
print("PC2 top:", loadings.iloc[1].abs().sort_values(ascending=False).head())
print("Explained ratios:", pca.explained_variance_ratio_[:5])
print("PC1+PC2 %:", (pca.explained_variance_ratio_[0]+pca.explained_variance_ratio_[1])*100)
cum=np.cumsum(PCA().fit(Xs).explained_variance_ratio_)
print("PCs >=80%:", np.searchsorted(cum,0.8)+1)

# ================== Q5 LINEAR REGRESSION ==================
print("\n=== Q5 Linear Regression (House Price) ===")
cal=fetch_california_housing(as_frame=True); X_cal,y_cal=cal.data[['MedInc']],cal.target
lr=LinearRegression().fit(X_cal,y_cal); y_pred=lr.predict(X_cal)
print("Intercept:", lr.intercept_,"Slope:", lr.coef_[0])
print("MSE:", mean_squared_error(y_cal,y_pred),"MAE:", mean_absolute_error(y_cal,y_pred),"RMSE:", mean_squared_error(y_cal,y_pred,squared=False))

# ================== Q6 SALARY DATA ==================
print("\n=== Q6 Salary_Data.csv ===")
try:
    sal=pd.read_csv("Salary_Data.csv"); Xs,ys=sal[['YearsExperience']],sal['Salary']
    lr_sal=LinearRegression().fit(Xs,ys)
    print("Intercept:", lr_sal.intercept_,"Slope:", lr_sal.coef_[0])
except Exception as e: print("Salary_Data.csv missing:", e)

# ================== Q7 MULTIPLE LINEAR REGRESSION ==================
print("\n=== Q7 Multiple Linear Regression ===")
np.random.seed(0); n=200
Age=np.random.randint(18,60,n); Height=np.random.normal(165,10,n); Weight=np.random.normal(70,12,n); TimeOnExercise=np.random.normal(30,8,n)
WeightLoss=0.12*TimeOnExercise-0.01*Age+np.random.normal(0,1,n)
df=pd.DataFrame({'Age':Age,'Height':Height,'Weight':Weight,'TimeOnExercise':TimeOnExercise,'WeightLoss':WeightLoss})
Xw,yw=df[['Age','Height','Weight','TimeOnExercise']],df['WeightLoss']
mlr=LinearRegression().fit(Xw,yw)
print("WeightLoss coeffs:", dict(zip(Xw.columns,mlr.coef_)))
Xh=cal.data[['MedInc','HouseAge','AveRooms']]; yh=cal.target
mlr2=LinearRegression().fit(Xh,yh)
print("House coeffs:", dict(zip(Xh.columns,mlr2.coef_)))

# ================== Q8 POLYNOMIAL REGRESSION ==================
print("\n=== Q8 Polynomial Regression ===")
X=np.linspace(-3,3,200).reshape(-1,1); y=X.ravel()**3+np.random.normal(0,3,200)
poly=PolynomialFeatures(3); Xp=poly.fit_transform(X); pr=LinearRegression().fit(Xp,y)
print("Poly coefs:", pr.coef_)
plt.scatter(X,y,s=6); xx=np.linspace(-3,3,200).reshape(-1,1); plt.plot(xx,pr.predict(poly.transform(xx))); plt.show()

# ================== Q9 LOGISTIC REGRESSION ==================
print("\n=== Q9 Logistic Regression ===")
Xtr,Xte,ytr,yte=train_test_split(X_canc,y_canc,test_size=0.3,random_state=0)
log=LogisticRegression(max_iter=1000).fit(Xtr,ytr); yp=log.predict(Xte)
cm=confusion_matrix(yte,yp); print("CM:\n",cm)
if cm.size==4:
    tn,fp,fn,tp=cm.ravel(); acc=(tp+tn)/(tp+tn+fp+fn)
    prec=tp/(tp+fp) if tp+fp>0 else 0; rec=tp/(tp+fn) if tp+fn>0 else 0
    f1=2*prec*rec/(prec+rec) if prec+rec>0 else 0
    print("Acc:",acc,"Prec:",prec,"Rec:",rec,"F1:",f1)
print(classification_report(yte,yp))

# ================== Q10 GRADIENT DESCENT ==================
print("\n=== Q10 Gradient Descent ===")
Xg=X_cal.values.flatten(); y=y_cal; Xg=(Xg-Xg.mean())/Xg.std(); Xb=np.vstack([np.ones_like(Xg),Xg]).T
def grad_desc(X,y,lr=0.01,epochs=1000):
    theta=np.zeros(X.shape[1]); m=len(y); hist=[]
    for i in range(epochs):
        grad=(1/m)*X.T.dot(X.dot(theta)-y); theta=theta-lr*grad
        if i%(epochs//5 or 1)==0: hist.append((i,mean_squared_error(y,X.dot(theta))))
    return theta,hist
for lr in [0.001,0.01,0.1]:
    theta,hist=grad_desc(Xb,y,lr,2000)
    print("lr",lr,"theta:",theta,"sample history:",hist[:3])

# ================== Q11 REGULARIZATION ==================
print("\n=== Q11 Regularization ===")
Xn=np.linspace(-3,3,200).reshape(-1,1); yn=Xn.ravel()**3+np.random.normal(0,4,200)
poly10=PolynomialFeatures(10); Xn10=poly10.fit_transform(Xn)
lr_over=LinearRegression().fit(Xn10,yn); mse_over=mean_squared_error(yn,lr_over.predict(Xn10))
lass=Lasso(0.1,max_iter=5000).fit(Xn10,yn); rid=Ridge(1.0).fit(Xn10,yn)
print("MSE overfit:",mse_over,"MSE Lasso:",mean_squared_error(yn,lass.predict(Xn10)),"MSE Ridge:",mean_squared_error(yn,rid.predict(Xn10)))
print("Lasso non-zero:",np.sum(lass.coef_!=0))

# ================== Q12 DECISION TREE ==================
print("\n=== Q12 Decision Tree ===")
Xtr,Xte,ytr,yte=train_test_split(X_diab,y_diab,test_size=0.3,random_state=0)
dtr=DecisionTreeRegressor(max_depth=4).fit(Xtr,ytr)
print(export_text(dtr,feature_names=list(X_diab.columns)))
plt.figure(figsize=(12,6)); plot_tree(dtr,feature_names=X_diab.columns,filled=True); plt.show()

# ================== Q13 NAIVE BAYES ==================
print("\n=== Q13 Naive Bayes ===")
try:
    loan=pd.read_csv("loan_data.csv").dropna()
    Xloan=loan.select_dtypes(include=[np.number])
    yloan=(loan['Loan_Status']=='Y').astype(int)
    Xtr,Xte,ytr,yte=train_test_split(Xloan,yloan,test_size=0.3,random_state=0)
    nb=GaussianNB().fit(Xtr,ytr); print("NB acc:",nb.score(Xte,yte))
except Exception as e: print("loan_data.csv missing:", e)

# ================== Q14 SVC ==================
print("\n=== Q14 SVC ===")
Xtr,Xte,ytr,yte=train_test_split(X_canc,y_canc,test_size=0.3,random_state=0)
svc=SVC(kernel='rbf').fit(Xtr,ytr); print("SVC acc:",svc.score(Xte,yte))

# ================== Q15 KNN ==================
print("\n=== Q15 KNN ===")
Xtr,Xte,ytr,yte=train_test_split(X_iris,y_iris,test_size=0.3,random_state=0)
knn=KNeighborsClassifier(3).fit(Xtr,ytr); print("KNN acc:",knn.score(Xte,yte))

print("\n=== DONE ALL QUESTIONS ===")



=== Q1 Feature Selection ===
Non-constant iris cols: Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)'],
      dtype='object')
Quasi-constant mask: [ True  True  True  True]
Duplicates (iris): []
Diabetes drop cols: []
Iris MI: {'sepal length (cm)': np.float64(0.5005752867852349), 'sepal width (cm)': np.float64(0.298200566636748), 'petal length (cm)': np.float64(0.9890186138889012), 'petal width (cm)': np.float64(0.9832532035174952)}
Diab MI: {'age': np.float64(0.0), 'sex': np.float64(0.02191093681353684), 'bmi': np.float64(0.17472741252046653), 'bp': np.float64(0.0640651701615722), 's1': np.float64(0.0664114764761905), 's2': np.float64(0.010111224995335455), 's3': np.float64(0.07208854375985307), 's4': np.float64(0.09934996144723351), 's5': np.float64(0.15004916552529401), 's6': np.float64(0.10378776297280945)}
Chi2 selected indices: [2 3]


TypeError: SelectKBest.__init__() takes from 1 to 2 positional arguments but 3 were given